In [1]:
#pip install gurobipy


In [5]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB

# Step 1: Define Input Data (Extended with more products)
data = {
    "Product": ["Shirts", "Gloves", "Pants", "Skirts", "Jackets", "Socks", "Sweaters", "Shawls", "Hoodies", 
                "Male Undergarments", "Female Undergarments", "Jeans", "T-Shirts", "Trousers", "Caps", "Coats"],
    "Labor_Hours": [1.5, 0.2, 4.8, 2.1, 7.7, 0.3, 2.4, 2, 6.6, 4, 4.3, 7.2, 2.0, 3.2, 1.0, 8.5],
    "Cloth_Used": [2, 1.5, 3.8, 4.7, 5.9, 1.5, 3.9, 5.3, 4.2, 2.3, 2, 3, 2.5, 3.0, 1.8, 6.0],
    "Selling_Price": [30, 25, 65, 70, 115, 15, 35, 45, 48, 15, 17, 40, 30, 45, 20, 155],
    "Unit_Variable_Cost": [20, 10, 25, 30, 35, 4, 18, 20, 25, 3, 2, 22, 12, 15, 7, 70],
    "Production_Type": ["In-house", "In-house", "Outsourced", "In-house", "Outsourced", "In-house", "Outsourced", 
                         "Outsourced", "In-house", "Outsourced", "In-house", "Outsourced", "In-house", "In-house", 
                         "In-house", "Outsourced"],
    "Advertised": [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
}

products_df = pd.DataFrame(data)

# Constants
ad_budget = 50000  # Advertising budget
ad_spend_per_product = 5000  # Advertising cost per product if advertised
in_house_cost_multiplier = 1.0  # In-house production cost multiplier
outsourced_cost_multiplier = 1.1  # Outsourced production cost multiplier

# Step 2: Define Objective Function
# Profit = Revenue - Costs
# Revenue = Selling Price * Demand
# Costs = Variable Cost + Advertising Cost
products_df["Demand"] = np.random.randint(500, 2000, len(products_df))  # Dummy demand
products_df["Revenue"] = products_df["Selling_Price"] * products_df["Demand"]
products_df["Production_Cost"] = np.where(
    products_df["Production_Type"] == "In-house",
    products_df["Unit_Variable_Cost"] * in_house_cost_multiplier,
    products_df["Unit_Variable_Cost"] * outsourced_cost_multiplier,
)
products_df["Ad_Spend"] = products_df["Advertised"] * ad_spend_per_product
products_df["Total_Cost"] = products_df["Production_Cost"] * products_df["Demand"] + products_df["Ad_Spend"]
products_df["Profit"] = products_df["Revenue"] - products_df["Total_Cost"]

# Step 3: Define Constraints
# Ad spend should not exceed total budget
# Ad spend allocation is binary (0 or 1)
# Production is limited by resources (e.g., labor hours, materials)

# Resources Constraints
labor_limit = 12000  # Total labor hours available (increased limit)
cloth_limit = 18000  # Total cloth available (increased limit)

# Demand Constraints (new)
max_demand_limit = [1500, 1000, 1200, 1000, 1500, 2000, 1800, 1400, 2000, 1200, 1000, 1600, 1700, 1500, 1200, 1000]  # Max Demand

# Advertising budget allocation constraints (new)
max_advertising_limit = [0.2, 0, 0.25, 0.3, 0.3, 0, 0, 0, 0.3, 0, 0, 0.25, 0, 0.2, 0, 0.3]  # % of ad budget per product

# Production constraints per product (new)
max_production_limit = [1200, 500, 800, 900, 1000, 2000, 1500, 1500, 1000, 1800, 1800, 1000, 1200, 1300, 900, 600]  # Production limits

# Step 4: Build Optimization Model using Gurobi
model = Model("Profit_Optimization")

# Decision Variables
produce = model.addVars(len(products_df), vtype=GRB.BINARY, name="Produce")  # Whether to produce the product
advertise = model.addVars(len(products_df), vtype=GRB.BINARY, name="Advertise")  # Whether to advertise the product

# Objective: Maximize Profit
model.setObjective(
    sum(
        produce[i] * (products_df.loc[i, "Profit"] + advertise[i] * products_df.loc[i, "Ad_Spend"])
        for i in range(len(products_df))
    ),
    GRB.MAXIMIZE,
)

# Constraints
# Labor hours constraint
model.addConstr(
    sum(produce[i] * products_df.loc[i, "Labor_Hours"] * products_df.loc[i, "Demand"] for i in range(len(products_df))) <= labor_limit,
    name="Labor_Limit",
)

# Cloth usage constraint
model.addConstr(
    sum(produce[i] * products_df.loc[i, "Cloth_Used"] * products_df.loc[i, "Demand"] for i in range(len(products_df))) <= cloth_limit,
    name="Cloth_Limit",
)

# Advertising budget constraint
model.addConstr(
    sum(advertise[i] * ad_spend_per_product for i in range(len(products_df))) <= ad_budget,
    name="Ad_Budget_Limit",
)

# Demand limit constraints (new)
for i in range(len(products_df)):
    model.addConstr(
        produce[i] * products_df.loc[i, "Demand"] <= max_demand_limit[i],
        name=f"Demand_Limit_{products_df.loc[i, 'Product']}",
    )

# Advertising allocation constraint (new)
for i in range(len(products_df)):
    model.addConstr(
        advertise[i] * ad_spend_per_product <= max_advertising_limit[i] * ad_budget,
        name=f"Ad_Allocation_Limit_{products_df.loc[i, 'Product']}",
    )

# Production capacity limit constraints (new)
for i in range(len(products_df)):
    model.addConstr(
        produce[i] * products_df.loc[i, "Demand"] <= max_production_limit[i],
        name=f"Production_Limit_{products_df.loc[i, 'Product']}",
    )

# Solve the model
model.optimize()

# Step 5: Display Results
if model.status == GRB.OPTIMAL:
    products_df["Optimized_Produce"] = [produce[i].x for i in range(len(products_df))]
    products_df["Optimized_Advertise"] = [advertise[i].x for i in range(len(products_df))]
    total_profit = model.objVal

    print("Optimized Production Plan:\n", products_df)
    print("\nTotal Optimized Profit:", total_profit)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 51 rows, 32 columns and 96 nonzeros
Model fingerprint: 0x4f0794b3
Model has 8 quadratic objective terms
Variable types: 0 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [2e+02, 1e+04]
  Objective range  [5e+03, 1e+05]
  QObjective range [1e+04, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+02, 5e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 51 rows and 32 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 82722 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.272200000000e+04, b